In [1]:
# imports

%load_ext autoreload
%autoreload 2

import torch
from denoising_diffusion_pytorch import Unet, GaussianDiffusion
from IPython.display import display, Markdown

/home/wmi/miniforge3/envs/medicraft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"

model = Unet(
    dim=64,
    dim_mults=(1, 2, 4, 8),
    channels=1,
)
model.to(device=DEVICE)
display(Markdown(f"Model loaded to __{DEVICE}__ device."))

Model loaded to __cuda__ device.

In [3]:
diffusion = GaussianDiffusion(
    model,
    image_size=128,
    timesteps=1000,  # number of steps
    # loss_type = 'l1'    # L1 or L2
)
diffusion.to(device=DEVICE)

GaussianDiffusion(
  (model): Unet(
    (init_conv): Conv2d(1, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (time_mlp): Sequential(
      (0): SinusoidalPosEmb()
      (1): Linear(in_features=64, out_features=256, bias=True)
      (2): GELU(approximate='none')
      (3): Linear(in_features=256, out_features=256, bias=True)
    )
    (downs): ModuleList(
      (0): ModuleList(
        (0-1): 2 x ResnetBlock(
          (mlp): Sequential(
            (0): SiLU()
            (1): Linear(in_features=256, out_features=128, bias=True)
          )
          (block1): Block(
            (proj): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm): GroupNorm(8, 64, eps=1e-05, affine=True)
            (act): SiLU()
          )
          (block2): Block(
            (proj): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm): GroupNorm(8, 64, eps=1e-05, affine=True)
            (act): SiLU()
          )
          (res_c

In [4]:
# training_images = torch.randn(8, 3, 128, 128)
# training_images = training_images.to(device=DEVICE)
# loss = diffusion(training_images)
# loss.backward()

In [5]:
# sampled_images = diffusion.sample(batch_size=4)

In [8]:
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Trainer, Trainer1D


trainer = Trainer(
    diffusion,
    "../../data/healthy_eyes",
    train_batch_size=1,
    train_lr=2e-5,
    train_num_steps=7000,  # total training steps
    gradient_accumulate_every=2,  # gradient accumulation steps
    ema_decay=0.995,  # exponential moving average decay
    amp=True,  # turn on mixed precision
    num_samples=4,  # number of samples to save
    calculate_fid=False,  # calculate FID during sampling
)

trainer.train()

  0%|          | 0/7000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

...

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

training complete


In [10]:
from pathlib import Path
import sys
from PIL import Image
from tqdm import tqdm

sys.path.append("../")
from datasets.preprocess import get_reference_eyes_paths

DATA_ROOT_PATH = Path("../../data")
EASY_CASES_PATH = DATA_ROOT_PATH / "easy_cases_with_fluid"

images_paths = get_reference_eyes_paths(EASY_CASES_PATH)
images_sizes = set()
for image_path in tqdm(images_paths):
    images_sizes.add(Image.open(image_path).size)

print(images_sizes)

100%|██████████| 274/274 [00:00<00:00, 4202.52it/s]

{(1024, 583), (1024, 437), (1381, 445)}


In [5]:
import os
from pathlib import Path

results_path = Path("../../results_16.12_2")


def keep_last_models(num_models: int = 10) -> None:
    # keep only last num_models models
    list_of_models = sorted(results_path.glob("*.pt"), key=os.path.getmtime)
    models_to_remove = list_of_models[:-num_models]
    for model in models_to_remove:
        model.unlink()


keep_last_models(3)

[PosixPath('../../results_16.12_2/model-9.pt'), PosixPath('../../results_16.12_2/model-10.pt')]
[]
0


In [27]:
from pathlib import Path
from tqdm import tqdm


def fix_files_names(dir_path: str):
    """
    Fix files names in dir_path to be in the format with same length of digits
    """
    dir_path = Path(dir_path)
    for file in tqdm(dir_path.glob("*")):
        name = file.name.split("_")

        new_name = f"{name[0]}_{name[1]:>010}"
        print(new_name)
        file.rename(file.parent / new_name)
        # break


fix_files_names("../../datasets/healthy_eyes_512x256_dataset")

2010it [00:00, 33662.02it/s]

image_000341.png
image_000894.png
image_001930.png
image_001542.png
image_000720.png
image_001684.png
image_001123.png
image_001655.png
image_001234.png
image_000390.png
image_001593.png
image_000437.png
image_000845.png
image_000219.png
image_001868.png
image_000678.png
image_001299.png
image_001629.png
image_000839.png
image_001248.png
image_000092.png
image_000172.png
image_001771.png
image_000961.png
image_000513.png
image_001310.png
image_001466.png
image_001814.png
image_000265.png
image_001007.png
image_000604.png
image_000043.png
image_001539.png
image_001158.png
image_001189.png
image_001776.png
image_000175.png
image_000095.png
image_001317.png
image_000514.png
image_000966.png
image_000262.png
image_001813.png
image_001461.png
image_000044.png
image_000603.png
image_001000.png
image_000893.png
image_001545.png
image_001937.png
image_000346.png
image_001124.png
image_000727.png
image_001683.png
image_001652.png
image_001594.png
image_000842.png
image_000430.png
image_001233.p